In [ ]:
# magic
%matplotlib inline

# imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
# from IPython.display import SVG, display # svg images
import graphviz as gv
import random as rand
from math import *

# my libraries
from mcr import *
from graph import *

# make figures large enough
from pylab import rcParams
rcParams['figure.figsize'] = (10, 10)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

In [3]:
from mcr import *

In [4]:
# Create an empty square
square = MCR()

In [5]:
# Add a couple of shapes to the square
shape1 = Polygon([(0.2,0.2), (0.2, 0.7), (0.7,0.7), (0.7,0.2)])
shape2 = Polygon([(0.5,0.15), (1.0,0.15), (.75, .87)])
shape3 = Polygon([(0.0, 0.2), (0.0, 0.5), (0.3, 0.5), (0.3,0.2)])

square.add_obstacle(shape1)
square.add_obstacle(shape2)
square.add_obstacle(shape3)

# Add a start and goal
square.start = (0.05, 0.05)
square.goal  = (0.95, 0.95)

In [6]:
# Display the MCR
square.show_obstacles()

In [7]:
square.overlapped_obstacles[3]

In [8]:
rr

In [164]:
import networkx as nx
from itertools import permutations

In [133]:
centroids = {}

#for o in square.obstacles:
#    centroids[o.label]=(o.centroid.x, o.centroid.y)

for o in square.overlapped_obstacles:
    centroids[o.label] = (o.centroid.x, o.centroid.y)

centroids['start'] = square.start
centroids['goal'] = square.goal
centroids

In [184]:
G = nx.Graph()

In [186]:
G.add_nodes_from(centroids.keys())

In [187]:
for n, p in centroids.items():
    G.node[n]['pos'] = p
    G.node[n]['label'] = n

In [188]:
for n in G.nodes():
    print(n)

In [144]:
pos = nx.get_node_attributes(G, 'pos')

In [145]:
pos = nx.get_node_attributes(G, 'pos')

In [148]:
labels = nx.get_node_attributes(G, 'label')
labels

In [202]:
nx.draw(G, pos)
#nx.draw_networkx_labels(G,pos,labels)
plt.show()

In [192]:
G.

In [203]:
edges = []
edges

In [204]:
current_obstacles = square.overlapped_obstacles
for x, y in combinations(enumerate(current_obstacles),2):
    label = x[1].label + y[1].label
    if x[1].intersects(y[1]) and label.count(",") >= 1: 
        edges.append((x[1].label, y[1].label))

In [205]:
edges

In [206]:
G.add_edges_from(edges)

In [208]:
square.obstacles

In [207]:
G.edges(data=True)

In [197]:
for e in G.edges():
    print(e)

In [209]:
end_edges = []
for o in square.obstacles:
    end_edges.append(('start', o.label))
    end_edges.append(('end', o.label))
end_edges

In [212]:
square.create_graph()

In [8]:
square.obstacles

In [ ]:
straws = MCR(svg='straws.svg')

In [10]:
straws.show_obstacles()

In [11]:
ten = list(filter(lambda x:x.label == '10', straws.overlapped_obstacles))[0]

In [12]:
ten

In [13]:
twenty = list(filter(lambda x:x.label == '20', straws.overlapped_obstacles))[0]

In [ ]:
twenty

In [ ]:
straws.show_bare_obstacles()

In [ ]:
straws.obstacles

In [ ]:
shapes = MCR(svg='shapes.svg')

In [ ]:
shapes.show_obstacles()

In [ ]:
shapes.obstacles

In [ ]:
for s in shapes.obstacles:
    print(s.exterior)

In [ ]:
f = random_MCR(10)
f.show_bare_obstacles(labels=True)

In [ ]:
f.show_obstacles(labels=True)

In [ ]:
tff = list(filter(lambda x: x.label=='3,4,5', f.overlapped_obstacles))[0]

In [ ]:
adj = [x.label for x in f.overlapped_obstacles if x.touches(tff)]

In [ ]:
adj

In [ ]:
field = Polygon([(0,0), (1,0), (1,1), (0,1)])
for o in f.obstacles:
    field -= o
field

In [ ]:
f.overlapped_obstacles += field

In [ ]:
g.adj

In [16]:
print(g.to_graphviz())

In [11]:
e = gv.Graph(g.to_graphviz())

In [17]:
e.render()

In [ ]:
for v in g.vertices:
    print(v)

In [18]:
print('tetraw\na{}'.format(4))

In [19]:
from shapely.ops import polygonize
lines = [((0, 0), (1, 1)), ((0, 0), (0, 1)), ((0, 1), (1, 1)), ((1, 1), (1, 0)), ((1, 0), (0, 0))]
print(list(polygonize(lines)))

In [20]:
for o in f.overlapped_obstacles:
    print(o.label, o.area)

In [ ]:
f.obstacles[15]

LINEARRING (0.2113448481212785 0.9466046287414669, 1.005515418272208 0.4142050839633877, 0.8390083350299757 0.07933384098729857, 0.0448377648790459 0.6117333857653777, 0.2113448481212785 0.9466046287414669)
LINEARRING (-0.06366559104925472 0.6404146283813359, 0.5618611925782399 0.2210589424844612, 0.3953541093360073 -0.1138024774577606, -0.2301726742914872 0.3055532084391142, -0.06366559104925472 0.6404146283813359)
LINEARRING (0.001884843595678431 0.8902827628468879, 0.330793820728756 0.8902827628468879, 0.330793820728756 0.4019959730368554, 0.001884843595678431 0.4019959730368554, 0.001884843595678431 0.8902827628468879)
LINEARRING (0.0008394711793535607 0.5344054408282402, 0.4505638901715245 0.2329172153183041, 0.3308555885663917 -0.007834511109750864, -0.1188688304257792 0.2936537144001851, 0.0008394711793535607 0.5344054408282402)
LINEARRING (0.3908675558102189 0.9949181475969535, 0.6544478539170819 0.6888645714785959, 0.3690900729057502 0.3575286702267356, 0.3908675558102189 0.99

In [ ]:
f = random_mcr(20)
f.show_obstacles()

In [23]:
type(r)

In [24]:
r

In [26]:
print(r)

POLYGON ((0.1173029066521962 0.8801660155006444, 0.1687907094626689 1, 0.3929960491517601 0.9857722665013623, 0.4762635237624896 0.8308039247065071, 0.3789565681034012 0.7030239904332851, 0.1969807819338728 0.7191511743632599, 0.1173029066521962 0.8801660155006444))


In [ ]:
rotations = MCR(svg='rotation.svg')
rotations.show_obstacles()

In [ ]:
MCR

In [ ]:
__debug__